# Анализ текстов
Какие проблемы возникают?
    1. Тексты грязные, с опечатками
    2. Как формировать признаки?
  

## Предобработка

    1. Приведение к нижнему регистру

In [1]:
def get_lower(text):
    return text.lower()

    2. Удаление пунктуации

In [2]:
def remove_punctuation(text):
    exclude = r'[!?,.:;()"*/\\_]'
    s = re.sub(exclude, ' ', text)
    s = re.sub('-', '', s)
    return ' '.join(s.split())

    3. Приведение к нормальной форме
Врача -> Врач, делали -> делать

In [4]:
import pymorphy2
MORPH = pymorphy2.MorphAnalyzer()
NORMAL_FORM = {}

# сначала научимся работать со словами
def get_word_normal_form(word):
    if word not in NORMAL_FORM:
        NORMAL_FORM[word] = MORPH.parse(word)[0].normal_form
    return NORMAL_FORM[word]

# а затем с текстом
def get_sentence_normal_form(sentence):
    return ' '.join(get_word_normal_form(word) for word in sentence.split())

    4. Что еще?...

## Переход к признакам
    1. Bag of words (CountVectorizer, HashingVectorizer)
    2. Tf-idf Vectorizer
    3. word2vec (doc2vec)

### 1. Bag of words
![](http://www.python-course.eu/images/document_representation.png)
CountVectorizer - просто считает вхождения разных слов. HashingVectorizer - то же самое, но размер выборки снижается за счет того, что некоторые значения кодируются одним и тем же хешом.

In [11]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

In [18]:
cat = ['talk.religion.misc','soc.religion.christian'] # Выбираем две категории для классификации
# Данные для обучения
twenty_train = fetch_20newsgroups(subset='train', categories = cat, shuffle=True, random_state=1
                                  ,remove=('headers', 'footers', 'quotes'))
# Данне для проверки
twenty_test = fetch_20newsgroups(subset='test', categories = cat, shuffle=True, random_state=1
                                  ,remove=('headers', 'footers', 'quotes'))

In [10]:
vectorizer = CountVectorizer() # можно задавать ограничения
vectors = vectorizer.fit_transform(newsgroups_train.data)

### 2. Tf-Idf (term frequency - inverse document frequency)
Идея в том, чтобы оценивать важность слова в пределах документа и понижать важность для общеупотребительных слов
![](http://www.joyofdata.de/blog/wp-content/uploads/2014/02/tf-idf.png)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

### 3. Word2vec
В кратце - с помощью нейросети обучить сжатое векторное представление слова на основе его сочетания(контекста) с другими словами

Есть готовые натренированные корпусы текстов, но можно обучить и на своих

http://nlpx.net/archives/179

http://mccormickml.com/assets/word2vec/Alex_Minnaar_Word2Vec_Tutorial_Part_I_The_Skip-Gram_Model.pdf

In [15]:
from gensim.models import word2vec
w2v_model = word2vec.Word2Vec.load_word2vec_format('/Users/admin/Downloads/', binary=True) 

NameError: name 'ds' is not defined

In [ ]:
# типичный пример

Таким образом можно получить векторные представления слов. А что делать с предложениями?

In [16]:
def get_embedding(sentence):
    words = list(filter(lambda x: x in w2v_model, sentence.split()))
    if len(words) == 0:
        return None
    vectors = list(map(lambda x: w2v_model[x], words))
    embedding = np.mean(vectors, axis=0) #max,median,min...
    embedding /= np.linalg.norm(embedding, axis=0)
    return embedding

Полезно делать  **взвешенное** по Tf-Idf среднее!

### Кое-что еще
    Хорошо работают рекуррентные/сверточные нейронные сети

# ToDo:
    1. Реализовать взвешенное по tf-idf word2vec
    2. Реализовать полный цикл препроцессинга слов
    3. Сравнить качество классификации по следующим моделям на логистической регрессии (по train/test):
        3.1. CountVectorizer
        3.2. HashingVectorizer(несколько разных размеров слов)
        3.3. Tf-Idf
        3.4. Word2Vec натренированный по коллекции слов документа (среднее/макс/минимум)
        3.5. Word2Vec корпус из интернета (среднее/макс/минимум)

In [ ]:
# если сложно с такими размерностями, то можно попробовать PCA
from sklearn.decomposition import PCA,SparsePCA,TruncatedSVD
pca = TruncatedSVD(n_components=100)
pca.fit(X)

In [ ]:
var_rat = pca.explained_variance_ratio_
#как мы видим какого-то колоссального отрыва у небольшого числа компонент нету
np.sort(var_rat)